                                    Variant 29 | Fedorov Andrey 3530903/70302

## Задание 4.2

Реализовать генетический алгоритмрешения задачи о рюкзаке, используя свой набор генетических операторов (определяется номером в списке и другими параметрами), и получить результаты на своем наборе данных (определяется номером в списке).

**1. Начальная популяция – кол-во особей всегда  = 200:**

    1.2 - жадный выбор, начиная со случайного груза

**2. Отбор особей для скрещивания:**

    2.2 - выбрать только 20% самых приспособленных особей

**3. Скрещивание (кроссинговер) между выбранными особями. Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка:**

    3.2 - однородный (каждый бит от случайно выбранного родителя)

**4. Мутация:**

    4.2 случайное изменение 3х битов у 5% особей

**5. Формирование новой популяции (кол-во особей - константа)**

    5.3 - замена своих родителей


In [1]:
import numpy
import random
import math

In [2]:
data=[]
weidhts=[]
volumes=[]
values=[]
file = open('data/29.txt')
for line in file:
    data.append(line.split())
maxVolume = float(data[0][1])
maxWeight = float(data[0][0])
del data[0]
for item in data:
    weidhts.append(float(item[0]))
    volumes.append(float(item[1]))
    values.append(float(item[2]))

indNum=200
population = []

In [3]:
indexes = list(range(len(data)))
indexedDictData = dict(zip(indexes, data))
list_Data = list(indexedDictData.items())
list_Data.sort(key=lambda i: float(i[1][2])) #сортировка по ценности
list_Data.reverse()

In [4]:
#1. Начальная популяция - жадный выбор
population = []
#для каждого индивида
for i in range(indNum):
    individual=''
    #заполняем нулями 30 грузов
    for j in range(len(data)):
        individual+='0'
    indWeight = 0
    indVolume = 0
    #начало со случайного
    tempRand = random.randrange(len(data))
    individual = individual[:tempRand] + '1' + individual[tempRand+1:len(individual)]
    indIndexes = [tempRand]
    indWeight = float(indexedDictData[tempRand][0])
    indVolume = float(indexedDictData[tempRand][1])
    #для каждого отсортированного груза
    for z in range(len(individual)):
        #если попали на тот, который выбран рандомно
        if list_Data[z][0] in indIndexes:
            continue;
        tempWeight = float(list_Data[z][1][0])
        tempVolume = float(list_Data[z][1][1])
        #если превышен макс. вес или макс. объем
        if indWeight + tempWeight >= maxWeight or indVolume + tempVolume >= maxVolume:
            break;
        else:
            indWeight+=tempWeight
            indVolume+=tempVolume
            indIndexes.append(list_Data[z][0])
            individual = individual[:list_Data[z][0]] + '1' + individual[list_Data[z][0]+1:len(individual)]
    population.append(individual)

In [5]:
#1. Начальная популяция - случайная генерация
#population = []
#for i in range(indNum):
#    individual=''
#    for j in range(len(data)):
#        individual+=(random.randrange(2))
#    population.append(individual)

In [6]:
def fitness(individual):
    weight = 0.0
    volume = 0.0
    value = 0.0
    for i in range(len(individual)):
        weight += int(individual[i])*weidhts[i]
        volume += int(individual[i])*volumes[i]
        value += int(individual[i])*values[i]
    if volume > maxVolume or weight > maxWeight:
        return maxWeight, maxVolume,  0           
    return weight, volume, value

In [7]:
#2. Отбор для скрещивания - 20% самых приспособленных
def selection(population):
    sortedPop = sorted(population, key=  lambda i: fitness(i)[2], reverse=True)
    indForCross=[]
    for i in range(math.floor(len(sortedPop)*0.2)):
        indForCross.append(sortedPop[i])
    return indForCross

In [8]:
#3. Скрещивание однородное (каждый бит от случайного родителя) Check
def crossingover(indForCross):
    children=[]
    parentsArr = [i for i in range(len(indForCross))]
    #print(parentsArr)
    for i in range(len(indForCross)):
        newBorn1=''
        newBorn2=''
        if i<len(indForCross)-i-1:
            for j in range(len(data)):
                if indForCross[i][j] == indForCross[len(indForCross)-i-1][j]:
                    bit=indForCross[i][j]
                    newBorn1+=bit
                    newBorn2+=bit
                else:
                    bit=random.randrange(2)
                    newBorn1+=str(bit)
                    bit=random.randrange(2)
                    newBorn2+=str(bit)
            children.append(newBorn1)
            children.append(newBorn2)
    return children

In [9]:
numBits = 3
#4. Мутация - случайное изменение 3-ёх битов у 5% особей
def mutation(children):
    childList = []
    #для 5% детей
    for i in range(math.floor(len(children)*0.05)): #5% в штуках
        childNum=random.randrange(len(children)) # наугад выбираем ребёнка
        while(childNum in childList):
            childNum=random.randrange(len(children))
        childList.append(childNum)
        
        bitList = []
        #для 3 битов
        for j in range(numBits):
            bitNum=random.randrange(len(children[i])) # наугад выбираем бит
            while(bitNum in bitList):
                bitNum=random.randrange(len(children[i]))
            bitList.append(bitNum)
            if(children[childNum][bitNum]=='1'):
                children[childNum] = children[childNum][:bitNum] +'0'+children[childNum][bitNum+1:len(children[i])]
            else:
                children[childNum] = children[childNum][:bitNum] +'1'+children[childNum][bitNum+1:len(children[i])]


    return children

In [10]:
#5. Формирование новой популяции - замена своих родителей
def createNewPopulation(population, parents, children):
    newPopulation=[]
    individEntry ={}
    for child in children:
        newPopulation.append(child)       
    for individ in population:
        individEntry[individ] = 0
    for individ in parents:
        individEntry[individ] +=1
    for individ in population:
        if individEntry[individ]==0:
            newPopulation.append(individ)
        else:
            individEntry[individ] -=1
    return newPopulation

In [11]:
def solver(population):
    lastIterRes=0
    for i in range(500):
        selectedInd=selection(population)
        children=crossingover(selectedInd)
        mutChildren=mutation(children)
        population=createNewPopulation(population, selectedInd, mutChildren)
        bestIndivid=sorted(population, key=  lambda i: fitness(i)[2], reverse=True)[0]
        bestIndividFit=fitness(bestIndivid)[2]
        if abs(bestIndividFit-lastIterRes) < math.floor(int(list_Data[len(list_Data)-1][1][2])):
            return bestIndivid
        if i>0 and i%15==0:
            lastIterRes=bestIndividFit
    return bestIndivid

In [12]:
best=solver(population)
print("Рюкзачок")
print("------------------")
for i in range(len(best)):
    if(best[i]=='1'):
        print(i, data[i])
print("------------------")
print("Total Weight:", fitness(best)[0])
print("Total Volume:", round(fitness(best)[1],2))
print("Total Value:", fitness(best)[2])
print("------------------")
print("Max weight:",maxWeight)
print("Max volume:",maxVolume)

Рюкзачок
------------------
3 ['369', '0.7', '368']
5 ['1360', '0.9', '336']
7 ['216', '1.2', '268']
9 ['946', '0.4', '367']
11 ['333', '0.9', '376']
13 ['817', '0.9', '345']
14 ['1430', '0.5', '329']
15 ['120', '0.9', '312']
21 ['519', '0.7', '376']
24 ['939', '0.7', '332']
25 ['1049', '0.5', '336']
26 ['1613', '0.5', '263']
27 ['578', '0.9', '371']
28 ['874', '1.1', '285']
29 ['1652', '0.8', '318']
------------------
Total Weight: 12815.0
Total Volume: 11.6
Total Value: 4982.0
------------------
Max weight: 13000.0
Max volume: 12.0
